# STAC Catalog Search

This notebook demonstrates how to search STAC catalogs using GeoAgent's catalog registry and search tools.

In [ ]:
# %pip install geoagent

## Browse Available Catalogs

In [ ]:
from geoagent.catalogs.registry import CatalogRegistry

reg = CatalogRegistry()
for cat in reg.list_catalogs():
    auth = "(auth required)" if cat.requires_auth else ""
    print(f"{cat.name:25s} {cat.url} {auth}")

## Search Earth Search (AWS)

Search for Sentinel-2 imagery over San Francisco:

In [ ]:
client = reg.get_client("earth_search")
print(f"Connected to: {client.title or client.id}")

results = client.search(
    collections=["sentinel-2-l2a"],
    bbox=[-122.5, 37.5, -122.0, 38.0],
    datetime="2024-07-01/2024-07-31",
    max_items=5,
)

items = list(results.items())
print(f"\nFound {len(items)} items:")
for item in items:
    print(f"  {item.id}: {item.datetime}")

## Inspect Item Assets

In [ ]:
if items:
    item = items[0]
    print(f"Item: {item.id}")
    print(f"Datetime: {item.datetime}")
    print(f"Bbox: {item.bbox}")
    print(f"\nAssets:")
    for name, asset in item.assets.items():
        print(f"  {name}: {asset.title or 'N/A'}")

## Search Planetary Computer

In [ ]:
pc_client = reg.get_client("planetary_computer")

results = pc_client.search(
    collections=["landsat-c2-l2"],
    bbox=[-105.3, 39.9, -104.9, 40.1],  # Denver, CO
    datetime="2024-06-01/2024-06-30",
    max_items=3,
)

items = list(results.items())
print(f"Found {len(items)} Landsat items near Denver:")
for item in items:
    cloud = item.properties.get("eo:cloud_cover", "N/A")
    print(f"  {item.id}: {item.datetime} (cloud: {cloud}%)")

## Add a Custom Catalog

In [ ]:
reg.add_catalog(
    name="my_stac",
    url="https://earth-search.aws.element84.com/v1",
    description="My custom STAC endpoint",
)

print(f"Total catalogs: {len(reg.list_catalogs())}")
cat = reg.get_catalog("my_stac")
print(f"Custom catalog URL: {cat.url}")